In [1]:
import ee

In [2]:
ee.Authenticate()

True

In [3]:
ee.Initialize(project='ee-imhayatkhan')

In [ ]:
import geemap


In [5]:
m = geemap.Map()
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

<IPython.core.display.Javascript object>

In [19]:
m = geemap.Map(center = [53.3811, 1.4701], zoom = 8) # UK
m

Map(center=[53.3811, 1.4701], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

In [22]:
m = geemap.Map(data_ctrl=False,toolbar_ctrl=False, search_ctrl=False, draw_ctrl=False, measure_ctrl=False, layer_ctrl=False, geocoder_ctrl=False,fullscreen_ctrl=False, home_ctrl=False, add_data_ctrl=False, print_ctrl=False, time_slider_ctrl=False, inspector_ctrl=False, add_marker_ctrl=False, add_polygon_ctrl=False, add_rectangle_ctrl=False, add_circle_ctrl=False, add_ee_layer_ctrl=False, add_text_ctrl=False, add_image_marker_ctrl=False, add_geojson_file_ctrl=False) # UK
m

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

# Adding Basemaps

In [24]:
m = geemap.Map(basemap = 'Esri.WorldImagery')
m.add_basemap('Esri.WorldTopoMap')
m.add_basemap('OpenTopoMap')
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [26]:
# Listing Available Basemaps
basemap = list(geemap.basemaps.keys())
print(f'Total basemaps Available: {len(basemap)}')

Total basemaps Available: 150


In [27]:
basemap[0:5] # First 5 Basemaps

['OpenStreetMap',
 'Esri.WorldStreetMap',
 'Esri.WorldImagery',
 'Esri.WorldTopoMap',
 'FWS NWI Wetlands']

# Interactive basemap Select

In [28]:
m = geemap.Map()
m.add('basemap_selector')
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [14]:
# The code has been copied to the clipboard. 
# Press Ctrl+V to in a code cell to paste it.
import math

START_DATE = '2022-10-01'
END_DATE = '2023-06-02'
CLOUD_FILTER = 60
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50

aoi = ee.FeatureCollection('FAO/GAUL/2015/level0').filterMetadata('ADM0_CODE','equals',256)

dw = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
.filterDate(START_DATE, END_DATE) \
.filterBounds(aoi)

classification = dw.select('label')
dwComposite = classification.reduce(ee.Reducer.mode())

# Clip the composite and add it to the Map.
croplands = dwComposite.updateMask(dwComposite.clip(aoi).eq(4))
grass = dwComposite.updateMask(dwComposite.clip(aoi).eq(2))
combinedMask = croplands.blend(grass)

def func_sdj(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(aoi) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY') \
    .filterBounds(aoi) \
    .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the '"system":index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(
        ee.Dictionary({
            'primary': s2_sr_col,
            'secondary': s2_cloudless_col,
            'condition': ee.Filter.equals(
                ee.Dictionary({
                    'leftField': 'system:index',
                    'rightField': 'system:index'
                })
            )
        })
    ))

get_s2_sr_cld_col = func_sdj

def func_jxh(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

add_cloud_bands = func_jxh

def func_bcm(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')))

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10) \
    .reproject(**{'crs': img.select(0).projection(), 'scale': 100}) \
    .select('distance') \
    .mask() \
    .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

add_shadow_bands = func_bcm

def func_rmu(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    #  Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    #  Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    #  Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    #  20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20) \
    .reproject(**{'crs': img.select([0]).projection(), 'scale': 20}) \
    .rename('cloudmask'))

    #  Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

add_cld_shdw_mask = func_rmu

def func_xmx(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

apply_cld_shdw_mask = func_xmx

s2_sr_cld_col_eval = get_s2_sr_cld_col(geometry, START_DATE, END_DATE)
l8toa = s2_sr_cld_col_eval.map(add_cld_shdw_mask).map(apply_cld_shdw_mask)

#roi = ee.Geometry.Point(-6.173543904111879, 8.062628227578857)
roi = geometry2 

Map.centerObject(roi, 10)

# This field contains UNIX time in milliseconds.
timeField = '"system":time_start'

# Use this function to mask clouds in Landsat 8 imagery.

def func_jyf(image):
    quality = image.select('BQA')
    cloud01 = quality.eq(61440)
    cloud02 = quality.eq(53248)
    cloud03 = quality.eq(28672)
    mask = cloud01.Or(cloud02).Or(cloud03).Not()
    return image.updateMask(mask)

maskClouds = func_jyf

# Use this function to add variables for NDVI, time and a constant
# to Landsat 8 imagery.

def func_phc(image):
    # Compute time in fractional years since the epoch.
    date = ee.Date(image.get(timeField))
    years = date.difference(ee.Date('1970-01-01'), 'year')
    # Return the image with the added bands.
    return image \
    .addBands(image.normalizedDifference(['B8', 'B4']).rename('NDVI')).float() \
    .addBands(ee.Image(years).rename('t').float()) \
    .addBands(ee.Image.constant(1))

addVariables = func_phc

# Remove clouds, add variables and filter to the area of interest.
filteredLandsat = l8toa \
.filterBounds(geometry) \
.map(addVariables)

# Plot a time series of NDVI at a single location.
l8Chart = ui.Chart.image.series(filteredLandsat.select('NDVI'), roi, ee.Reducer.mean(), 30) \
    .setChartType('ScatterChart') \
    .setOptions({
        "title": 'Landsat 8 NDVI time series at ROI',
        "trendlines": {"0": {
            "color": 'CC0000'
        }},
        "lineWidth": 1,
        "pointSize": 3,
    })
print(l8Chart.getInfo())

# Linear trend ----------------------------------------------------------------
# List of the independent variable names
independents = ee.List(['constant', 't'])

# Name of the dependent variable.
dependent = ee.String('NDVI')

# Compute a linear trend.  This will have two "bands": 'residuals' and
# a 2x1 band called coefficients (columns are for dependent variables).
trend = filteredLandsat.select(independents.add(dependent)) \
.reduce(ee.Reducer.linearRegression(independents.length(), 1))
# Map.addLayer(trend, {}, 'trend array image')

# Flatten the coefficients into a 2-band image
coefficients = trend.select('coefficients') \
.arrayProject([0]) \
.arrayFlatten([independents])

# Compute a de-trended series.

def func_fhc(image):
    return image.select(dependent).subtract(
    image.select(independents).multiply(coefficients).reduce('sum')) \
    .rename(dependent) \
    .copyProperties(image, [timeField])

# Apply the de-trending function to the filtered Landsat data.
detrended = filteredLandsat.map(func_fhc)

# Plot the detrended results.
detrendedChart = ui.Chart.image.series(detrended, roi, None, 30).setOptions(
    {
        "title": 'Detrended Landsat time series at ROI',
        "lineWidth": 1,
        "pointSize": 3,
    }
)
print(detrendedChart.getInfo())

# Harmonic trend ----------------------------------------------------------------
# Use these independent variables in the harmonic regression.
harmonicIndependents = ee.List(['constant', 't', 'cos', 'sin'])

# Add harmonic terms as new image bands.

def func_rhv(image):
    timeRadians = image.select('t').multiply(2 * math.pi)
    return image \
    .addBands(timeRadians.cos().rename('cos')) \
    .addBands(timeRadians.sin().rename('sin'))

harmonicLandsat = filteredLandsat.map(func_rhv)

# The output of the regression reduction is a 4x1 array image.
harmonicTrend = harmonicLandsat \
.select(harmonicIndependents.add(dependent)) \
.reduce(ee.Reducer.linearRegression(harmonicIndependents.length(), 1))

# Turn the array image into a multi-band image of coefficients.
harmonicTrendCoefficients = harmonicTrend.select('coefficients') \
.arrayProject([0]) \
.arrayFlatten([harmonicIndependents])

# Compute fitted values.

def func_bod(image):
    return image.addBands(
    image.select(harmonicIndependents) \
    .multiply(harmonicTrendCoefficients) \
    .reduce('sum') \
    .rename('fitted')).setOptions({
    "title": 'Harmonic model: original and fitted values',
    "lineWidth": 1,
    "pointSize": 3,
})
# Plot the fitted model and the original data at the ROI.
fittedHarmonic = harmonicLandsat.map(func_bod)

harmonicChart = ui.Chart.image.series(
    fittedHarmonic.select(['fitted', 'NDVI']), roi, ee.Reducer.mean(), 30
).setSeriesNames(['NDVI', 'fitted']).setOptions({
    "title": 'Harmonic model: original and fitted values',
    "lineWidth": 1,
    "pointSize": 3,
})

print(harmonicChart.getInfo())

# Use the HSV to RGB transform to display phase and amplitude
rgb = phase.unitScale(-math.pi, math.pi).addBands(
amplitude.multiply(2.5)).addBands(
filteredLandsat.select('NDVI').median()).hsvToRgb()
rgb = rgb.updateMask(combinedMask)

Map.addLayer(l8toa.filterDate('2023-05-01','2023-06-01').median(), {'bands':['B4','B3','B2'], "min":0, "max":3000}, 'Sentinel 2', False)
Map.addLayer(l8toa.filterDate('2019-05-01','2019-06-01').median(), {'bands':['B4','B3','B2'], "min":0, "max":0.3}, 'Landsat 8', False)
Map.addLayer(rgb, {}, 'phase (hue), amplitude (saturation)')

# use unit scale to normalize the pixel values
rgb = rgb.unitScale(0, 1)

eightbitRGB = rgb.multiply(255).toByte()

print(eightbitRGB.getInfo())

Map.addLayer(eightbitRGB, {"min":0, "max":255}, 'rgb_')

image_export_options = {
    'patchDimensions': [768, 768],
    'maxFileSize': 104857600,
    'kernelSize': [256, 256],
    'compressed': True
}
geometry = ee.FeatureCollection('FAO/GAUL/2015/level0').filterMetadata('ADM0_CODE','equals',256)
Export.image.toDrive(
    image=eightbitRGB.clip(geometry),
    description="Export_Task",
    folder='extra_county',
    fileNamePrefix="extra_county",
    region=geometry,
    scale=10,
    crs="EPSG:27700",
    maxPixels=29073057936,
    formatOptions=image_export_options,
    fileFormat='TFRecord'
)


NameError: name 'geometry' is not defined

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…